# Explaining MBL player Salaries vs. Performance

# 1. Objective, Dataset and Analytical Approach


## 1.1 The Objective
In this project we would like to do some analysis to find out if baseball players are fairly compensated and if their performance is reflected in their compensation. Are there players that get disproportionately compensated compared to their peers? Are some overpaid or underpaid? We want to see if there are big outliers when comparing performance and salary. Furthermore, we want to find out if salaries increase linear with performance or exponentially. Are superstars being worth their money? 


## 1.2 The Dataset
To answer the above questions, we will work with a database with historical data from the MBL. The data originally came from [Sean Lahman's site](http://www.seanlahman.com/baseball-archive/statistics/), and was transformed to a [sqlite database](https://s3.amazonaws.com/dq-content/224/lahman2015.sqlite).

The database contains a number of tables with statistics on baseball players from 1871 to 2015.
The main tables that contain the data that we need are:
 - Master -- contains information about players (including names)
 - Batting -- contains information on batting by year.
 - Pitching -- information on pitching by year.
 - Fielding -- information on fielding by year.
 - Salaries -- salary data for each player by year, starting in 1985.


## 1.3 The Analytical Approach
To analyze if the players are compensated fairly based on their performance need two pieces of data, the player salary and the player performance. For this analysis we will only use the data of one year to ensure comparability. 

The salary is available in a database for each player. However the performance is a bitg more tricky to judge. The good news is that the database contains all information that is relevant to a players performance and it should allow us to construct a performance score/index. Unfortunately I am not a Baseball expert - hence I don't know which metric one would look at to judge a good player from a bad one. But this is not problem as we can leverage data science to solve for this issue. We can find out which features of a player correlate the strongest with player salaries. We can take the to 5 and create a weighted average that represents the player performance score.

Once we have the performance for each player, we can look at the corellation between performance and salary. The most telling analysis will be the scatter chart of salary over performance where we can observe how well the dots line up.

In short, we will perform the following analytical steps:
1. Connect to the MBL player database and get take an initial look at the player salaries 
1. Correlate player features with salaries and identify the most significant features
2. Define and compute a player performance score based on the most significant features
3. Correlate salary and performance

In order for the data to be comparable, we will only use the data from most recent year which is 2015. You can easily conduct the analysis for other years as well and even see if the pay for performance has changed over the years.


# 3. The Analysis

## 3.1 Connect to the Database and Initial Analysis

As a first step we need to connect to the SQLite3 database using the Python sqlite3 package. We can then take an initial look at some of the tables. For a full description of the tables please refer to this [document](http://seanlahman.com/files/database/readme2016.txt) as we will not explain each column of each table.

In [ ]:
#Import sqlite3 
import sqlite3

#connect to the database and create a cursor
conn = sqlite3.connect("lahman2015.sqlite")
cursor = conn.cursor()

#Run an SQL query to return the  5 youngest players by year of birth
query = "SELECT * FROM master ORDER BY birthYear DESC LIMIT 5;"
cursor.execute(query)
print (cursor.fetchall())


We can see that the youngest players in the season year 2015 was from 1995 - so 20 years old.

Next let's look at the salaries in the year 2015 - and who were the 10 higherst earners, the 10 lowerst earners and the average salary. For this we need to execute a joint with the `Master` table and the `Salaries` table.

In [ ]:
#Define and execute and print sql query for the top 10 players by salary
query = "SELECT Master.nameFirst, Master.nameLast, Salaries.salary FROM Master INNER JOIN Salaries ON Master.playerID == Salaries.playerID WHERE Salaries.yearID = 2015 ORDER BY Salaries.salary DESC LIMIT 10;"
cursor.execute(query)
print ("Top 10 MBL players by Salary in 2015")
print (cursor.fetchall())

#Define, execute and print sql query for the bottom 10 players by salary
query = "SELECT Master.nameFirst, Master.nameLast, Salaries.salary FROM Master INNER JOIN Salaries ON Master.playerID == Salaries.playerID WHERE Salaries.yearID = 2015 ORDER BY Salaries.salary ASC LIMIT 10;"
cursor.execute(query)
print ("\nBottom 10 MBL players by Salary in 2015")
print (cursor.fetchall())

#Define, execute and print sql query for the average salary
query = "SELECT AVG(Salaries.salary) FROM Master INNER JOIN Salaries ON Master.playerID == Salaries.playerID WHERE Salaries.yearID = 2015;"
cursor.execute(query)
print ("\nAverage salary of players in 2015")
print (cursor.fetchall())

We can see that it pays well to play for the MBL. The top players earn USD 30m and more, the average player gets USD 4.2m and the lowest salaries are still round USD 500k. A similar analysis for Data Scientist salaries would certainly result into lower numbers. Also the variance is significant. The top paid player earns more than 60 time what the bottom player earns. Does performance really justify such a difference?

## 3.2 Correlate Player Features

Let's understand player performance. To do so, we need to get all players, their features and their salaries in one table. This requires a join of the tables `Master`, `Batting`, `Pitching`, `Fielding` and `Salaries` tables. We will do this in Pandas because it is easier to clean up the data and do some basic calculations

Each of the tables except for `Master` and `Salaries` contains statistics about the respective roles. For example, the Batting table has columns for the number of runs, hits, doubles, triples, homeruns, etc. of the player. I would expect that many of the features have little to not correlation to the player salary, such as how many times a player was at bats. But we will see.


In [ ]:
#Import Pandas
import pandas as pd

#First, we query the required data from the different tables and load it into pandas dataframes
year = "2015" #Analysis for 2015 - feel free change if you want to look at other years

#Master table with salaries
#The master table contains all players since 1871, but since we join it with salaries,
#we can filter out only the players that had a salary in the year that we analyze (2015)
query = "SELECT * FROM Master JOIN Salaries ON Master.playerID==Salaries.PlayerID WHERE yearID=" + year
cursor.execute(query)
master = pd.DataFrame(cursor.fetchall())
cols = ["m_"+element[0] for element in cursor.description] #get column names and add to dataframe - add m_ for master
cols[27] = "playerID2" #need to ensure we have only one playerID as we want to merge on this column later
master.columns=cols

#Load Batting table for year 2015
query = "SELECT * FROM Batting Where yearID=" + year
cursor.execute(query)
batting = pd.DataFrame(cursor.fetchall())
cols = ["b_" + element[0] for element in cursor.description] #get column names and add to dataframe, add b_ for batting
batting.columns=cols

#Load Pitching table for year 2015
query = "SELECT * FROM Pitching Where yearID=" + year
cursor.execute(query)
pitching = pd.DataFrame(cursor.fetchall())
cols = ["p_" + element[0] for element in cursor.description] #get column names and add to dataframe, add p_ for pitching
pitching.columns=cols

#Load Fielding table for year
query = "SELECT * FROM Fielding Where yearID=" + year
cursor.execute(query)
fielding = pd.DataFrame(cursor.fetchall())
cols = ["f_" + element[0] for element in cursor.description] #get column names and add to dataframe, add f_ for fielding
fielding.columns=cols

#Each player has a number of stints per season. Since we want to work with the totals of the players metrics, we need to add up
#the date of all stints
batting = batting.groupby("b_playerID").sum()
batting.reset_index(inplace=True) #take playerID out of the index
pitching = pitching.groupby("p_playerID").sum()
pitching.reset_index(inplace=True) #take playerID out of the index
fielding = fielding.groupby("f_playerID").sum()
fielding.reset_index(inplace=True) #take playerID out of the index

#Next, we need to merge the data-sets into one dataframe, this is done in several steps as pandas does not support
#multiple merge operations
print ("Shape of master/salaries: ", master.shape)
data = pd.merge(master, batting, left_on="m_playerID", right_on="b_playerID", how='outer')
print ("Shape after merging batting table: ", data.shape)
data = pd.merge(data, pitching, left_on="m_playerID", right_on="p_playerID", how='outer')
print ("Shape after merging pitching table: ", data.shape)
data = pd.merge(data, fielding, left_on="m_playerID", right_on="f_playerID" , how='outer')
print ("Shape after merging fielding table", data.shape)
print ("column fields:\n", data.columns)

Two things we can observer:
1. We now have 84 features/columns we can use for our analysis. A number of features are duplicated in the various tables and hence their names have been amended - e.g. G=Games is is translated to G_x, G_y and G. We have to keep this in mind when we evaluate the results
2. We  see that we don't have salary information available for all the batting, pitching and fielding players as the total row count is lower than the number of rows in the salary table.

In the following, we are only interested in players where we know the salary. Hence we will drop the rows where no salary is available.

In [ ]:
#drop rows without salary information and show shape after
data.dropna(subset=["m_salary"],inplace=True)
print("shape of data:", data.shape)

Now we have a dataframe with all features of the MBL players as well as their salary. Time to find out which feature is the most detrimental for the salary.

In [ ]:
#Correlate features with salary
r2_salary = data.corr()["m_salary"].sort_values(ascending=False)

#Plot the result as a bar chart using Bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

#Group all data necessary for the plot 
source = ColumnDataSource(data=dict(values=r2_salary.values.tolist(),
                                    features=r2_salary.index.tolist(),
                                    y=[i+0.5 for i in range(0,62)]))

#Configure the hover tool tips
hover = HoverTool(tooltips=[
    ("Feature", "@features"),
    ("Value", "@values"),
])

#Setup the figure
p = figure(plot_width=600, plot_height=1200, y_range=r2_salary.index.tolist(),
           title="Correlation of the player features with their Salary",
          tools=[hover])

#Draw the graph
p.hbar(y='y', height=0.5, left=0, color="navy", right='values', source=source)
show(p)


What you see in the graph is the corellation of the different player features with their salary. You can hover over the respective bar to see the value of the correlation.
Here some observations:
- Most features don't have a very strong positive or negative correlation
- Highest(negative) correlation is with the player birth year - which shows that the years of age/experience have a major impact on player salaries. Younger players just earn less than their older colleagues.
- Other than that, pitching performance (_p) seems to have a strong impact on salary - batting is only 2nd
- For fielding the only feature that seem half way relevant is CS (`Opponents Caught Stealing (by catchers)`)
- Correlation of salary with itself is 1 - which was to be expected
- The correlation with year ID is 0 - also not a surprise because the year is the same (2015) for all players

Now that we understand the correlation of salary to the player features, we can start to build a performance score.

## 3.3 Player Performance Score
To develop the performance score, we will take the 5 features that are most significant for player salary and create a weighted average. Let's look at the features that came out highest.
1. m_birthYear: The birth year of the player floats up with a correlation of -0.409. Whilst this is easy to esplain with the player experience and the players climbing up the salary ladder over time, it is not a great indicator of performance. So we will not include this in the score and better look down the list
2. `p_IPouts`: If we look this up [in the readme document](http://seanlahman.com/files/database/readme2016.txt), we can see that this refers to `Outs Pitched (innings pitched x 3)` in the pitching table (indicated by `p_` remembert we introduced prefixes `m_`, `b_`, `p_`, `f_` for the different tables so we can easier refer to the source). This seems to be a relevant performance score.
3. `p_GS`: `Games Started` also a relevant feature. Not sure what exactly this means as I am not a baseball expert, but I guess if you are setup to start the game, you would be considered high performance by the coach
4. `p_BFP`: `Batters faced by Pitcher` [Wikipedia](https://en.wikipedia.org/wiki/Batters_faced_(baseball)) says that this is the number of batters who made a plate appearance before the pitcher in a game or in a season. This seems to be an interesting feature as I would imagine that good pitchers can beat more batters. 
5. `p_SO`: `Strikeouts` can be looked up on [Wikipedia](https://en.wikipedia.org/wiki/Strikeout). In baseball, a strikeout (or strike-out) occurs when a batter racks up three strikes during a time at bat. It usually means the batter is out. This is very relevant for performance. As a coach we want to pay high salaries for players that do strikeouts.
6. `p_W`: `Wins` - This does not require much explanation. This is what we pay players for - the ultimate performance score

So we will use the features 2-6 above to create our score. If you want, you can add more features in the score to make it more accurate. Since their correlation is between 0.39 and 0.38 - there is no much difference in their significance. Hence, we will not weight the features but use a simple average to create the performance score. But before doing so, there are two more steps. First, we should to visualize the correlations in a scatter chart to see how much the features really correlate. After that we need to normalize the feature into a sub-score. This we can do by dividing the actual feature values by the maximum that a player has achieved in that category. Then we multiply by 100 to arrive at a score between 0 an 100. The values should all be positive - so we don't need to deal with negative values.

Let's get on this starting with the visualization.


In [ ]:
#First put all chart labels into a dictionary and extract the salary
features = {'p_IPouts': "Outs Pitched (innings pitched x 3)",
            'p_G': "Games Started",
            'p_BFP': "Batters faced by Pitcher",
            'p_SO' :"Strikeouts",
            'p_W': "Wins"}



#Plot 5 scatter diagrams with one feature each
from bokeh.layouts import gridplot
from bokeh.models.tickers import FixedTicker

p = []

for f_code, f_title in features.items():
    p.append (figure(plot_width=450, plot_height=450, title=f_title + " over Salary", x_axis_type="log")) 
    p[-1].circle(x=data['m_salary']/1000000, y=data[f_code], size=2, color='navy') #plotting salary in millions for easier readability
    p[-1].xaxis.axis_label = 'Salary in USDm (log)'
    p[-1].yaxis.axis_label = f_title
    p[-1].xaxis.ticker = FixedTicker(ticks=[0.5,1, 5, 10, 15,20,25, 30,35])



# make a grid
grid = gridplot([[p[0],p[1]], [p[2], p[3]], [p[4], None]])

# show the results
show(grid)




The above charts show the five features over the salary in a scatter plot. Please note that the salary is on a logarithmic scale - as I was expecting the salary to go up exponentially with improvement in the respective features. Also, the amount on the axis is shown in millions as otherwise the tick labels get a bit difficult to read 

What we see is that the correlation is not great, but if you take out the players at the low end of the salary range. There seems to be a minimum salary level that is payed once a player makes it to the MBL level - regardless of their performance. They would then have to work their way up. I would also imagine that the features explain the salary only in combination. Salaries only increase if you show performance on more than one feature. Hopefully this will be explained in our combined performance score.

No let's generate a performance score. To do that, we first need to normalize the data-sets so we can compare them and add them into an overall score. After that, we calculate the average of the normalized features as we don't apply a weight. Finally, we plot the performance score over the salary.

In [ ]:
#Normalize by deducting the min and deviding by the max value
#addding normalized values as a new collumn to `data`

for f in features:
    data['norm_' + f]= ( data[f] - data[f].min() ) / (data[f].max())

#Doing the same for the salary
data['norm_m_salary']= ( data['m_salary'] - data['m_salary'].min() ) / (data['m_salary'].max())

#Create the performance score as average
data["performance_score"] =  ( data['norm_p_BFP'] + data['norm_p_G'] + data['norm_p_IPouts'] + data['norm_p_SO'] + data['norm_p_W'] ) / 5

p = figure(plot_width=700, plot_height=700, title="Player Performance vs. Salary", x_axis_type="log")
p.circle(x=data['norm_m_salary'], y=data['performance_score'], size=2, color='navy') 
p.xaxis.axis_label = 'Normalized Salary'
p.yaxis.axis_label = 'Performance Score'

show(p)




# 4. Conclusion and Result

In this chart the horizontal axis with the salary is on a logarithmic scale again. This time normalized however.

At first look, we observe that there does not seem to be a strong correlation between the player salary and the performance. The dots on the chart look very scattered. However, if you look more carefully, you can discover that there are actually two correlations. On the left the players with lower salary get paid better if they perform better. On the right there is a group of players with high salaries. They also tend to get more salary if they perform better.

So we have a two class society amongst players here. There are the working horses as I would call them, that work hard to get a slight increase and there are the celebrities that get payd better by an orde of magnitude regardless of performance and they can even increase their salary if they perform well. This is not really fair.

This is an interesting finding and can be translated into insights for players and for club owners.

For players it means that performance is rewarded however if you want to make a quantum leap in your pay, you need to become some kind of celebrity and become part of the club on the right side of the chart. We could do more analysis to find out what distinguishes these groups and on which features they perform differently so that it sets them apart.

For club owners this result means that they should be able to hire great players at low prices. In fact, they would be better off using data science when hunting for their next player acquisition to get a better bang for the buck. Also, some players seem to be very overpaid and don't live up to the expectations that come with such a high salary.

Disclaimer:
As usual, please look at this analysis with a pinch of salt. While this gives us an indication, the analysis was not fully comprehensive. For example, we only looked at a sub-set of features. Performance might not be holistically defined by the 5 features we picked. You would expect high performing and highly paid players to perform well across all dimensions. Furthermore, some players might be valuable less because of their performance but because they are celebrities and bring in big advertising dollars. I would not recommend to make player purchase decision based on this analysis - but I would certainly recommend to use data science before your next player purchase.